# Importing VTK data to be displayed as Mesh

Additional requirements for this example: `vtk`

We will also see how to do some processing with `vtk` and visualize its results.

In [ ]:
import k3d
import os
import vtk
import numpy as np

from k3d.helpers import download

filename = download('https://raw.githubusercontent.com/naucoin/VTKData/master/Data/cow.vtp')

model_matrix = (
     1.0,  0.0, 0.0, 0.0,
     0.0,  0.0, 1.0, 0.0,
     0.0,  1.0, 0.0, 0.0,
     0.0,  0.0, 0.0, 1.0
)

reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName(filename)
reader.Update()

plot = k3d.plot()
cow3d = k3d.vtk_poly_data(reader.GetOutput(), color=0xff0000, model_matrix=model_matrix)
plot += cow3d
plot.display()

In [ ]:
f = open('./output.html', 'w', encoding='UTF-8')
f.write(plot.get_snapshot(9))
f.close()

plot.snapshot_include_js = False
f = open('./smaller_output.html', 'w', encoding='UTF-8')
f.write(plot.get_snapshot(9))
f.close()

In [ ]:
import os
print(os.stat('./output.html').st_size / 1024)
print(os.stat('./smaller_output.html').st_size / 1024)

In [ ]:
cow3d.wireframe = True

In [ ]:
cow3d.wireframe = False

In [ ]:
plane=vtk.vtkPlane()
plane.SetOrigin(0,0,0)
plane.SetNormal(1,1,0)
 
cutter=vtk.vtkCutter()
cutter.SetCutFunction(plane)
cutter.SetInputConnection(reader.GetOutputPort())
cutter.Update()

FeatureEdges = vtk.vtkFeatureEdges()
FeatureEdges.SetInputConnection(cutter.GetOutputPort())
FeatureEdges.BoundaryEdgesOn()
FeatureEdges.FeatureEdgesOff()
FeatureEdges.NonManifoldEdgesOff()
FeatureEdges.ManifoldEdgesOff()
FeatureEdges.Update()

cutStrips = vtk.vtkStripper() ; #Forms loops (closed polylines) from cutter
cutStrips.SetInputConnection(cutter.GetOutputPort())
cutStrips.Update()

cutPoly = vtk.vtkPolyData() ; #This trick defines polygons as polyline loop
cutPoly.SetPoints((cutStrips.GetOutput()).GetPoints())
cutPoly.SetPolys((cutStrips.GetOutput()).GetLines())

In [ ]:
model_matrix = (
     1.0,  0.0, 0.0, 0.0,
     0.0,  0.0, 1.0, -5.0,
     0.0,  1.0, 0.0, 0.0,
     0.0,  0.0, 0.0, 1.0
)
plot += k3d.vtk_poly_data(cutPoly, color=0x0000ff,  model_matrix=model_matrix)